In [5]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as io
import pandas as pd
from pandas import DataFrame as dframe
import cvxpy as cv

In [6]:
#対象被験者の特徴ベクトルをMATLABから入手
feat_C1 = np.loadtxt("feat_left_A02T_K0.txt",delimiter=",")
feat_C2 = np.loadtxt("feat_right_A02T_K0.txt",delimiter=",")
feat_Test_C1 = np.loadtxt("feat_test_left_A02T.txt",delimiter=",")
feat_Test_C2 = np.loadtxt("feat_test_right_A02T.txt",delimiter=",")

In [7]:
#非対象被験者の特徴ベクトルをMATLABから入手
feat_C1_K1 = np.loadtxt("feat_left_A01T.txt",delimiter=",")
feat_C2_K1 = np.loadtxt("feat_right_A01T.txt",delimiter=",")
feat_C1_K2 = np.loadtxt("feat_left_A03T.txt",delimiter=",")
feat_C2_K2 = np.loadtxt("feat_right_A03T.txt",delimiter=",")
feat_C1_K3 = np.loadtxt("feat_left_A05T.txt",delimiter=",")
feat_C2_K3 = np.loadtxt("feat_right_A05T.txt",delimiter=",")
feat_C1_K4 = np.loadtxt("feat_left_A06T.txt",delimiter=",")
feat_C2_K4 = np.loadtxt("feat_right_A06T.txt",delimiter=",")

In [8]:
def Dic(feat_l_tr,feat_r_tr): #入力の配列は2次元配列である必要がある
    feat_tr = np.concatenate([feat_l_tr, feat_r_tr],0)
    return feat_tr

In [9]:
#特徴ベクトルを2次元に変換
#特徴ベクトル g は２M×Tになっている必要がある
#Tは被験者の試行回数である
feat_tr = Dic(feat_C1,feat_C2)
#非対象被験者の特徴ベクトルも同様の処理を行う
feat_tr_K1 = Dic(feat_C1_K1,feat_C2_K1)
feat_tr_K2 = Dic(feat_C1_K2,feat_C2_K2)
feat_tr_K3 = Dic(feat_C1_K3,feat_C2_K3)
feat_tr_K4 = Dic(feat_C1_K4,feat_C2_K4)
#それぞれの特徴ベクトルをつなげる
feat_tr_dic_1 = Dic(feat_tr,feat_tr_K1)
feat_tr_dic_2 = Dic(feat_tr_dic_1,feat_tr_K2)
feat_tr_dic_3 = Dic(feat_tr_dic_2,feat_tr_K3)
feat_tr_dic = Dic(feat_tr_dic_3,feat_tr_K4).T
#feat_tr_dicはすべての特徴ベクトルをつなげたもの
#まず、分類に用いるテストデータの特徴ベクトルを2次元に変換
feat_l_te = feat_Test_C1;
feat_r_te = feat_Test_C2;
feat_l_te = feat_l_te.T
feat_r_te = feat_r_te.T
print(feat_l_te.shape,feat_tr_dic.shape,feat_tr.shape,feat_tr_dic_1.shape)
matrix_index = np.array([feat_C1.shape[0],feat_C2.shape[0],feat_C1_K1.shape[0],feat_C2_K1.shape[0],feat_C1_K2.shape[0],feat_C2_K2.shape[0],
                        feat_C1_K3.shape[0],feat_C2_K3.shape[0],feat_C1_K4.shape[0],feat_C2_K4.shape[0]])

(4, 40) (4, 200) (40, 4) (80, 4)


In [12]:
#各クラスの特徴ベクトルの配列の列数を格納した配列を引数にすればもう少し拡張性が上がる
def SRC(feat_tr,feat_tr_dic_1,feat_tr_dic_2,feat_tr_dic_3,feat_tr_dic,feat_l_te,matrix_index):
    answer = np.zeros(feat_l_te.shape[1])
    SRC_Cla = np.zeros((feat_tr_dic.shape[1], feat_l_te.shape[1]))
    SRC_l_Cla = np.zeros((feat_tr_dic.shape[1], feat_l_te.shape[1]))
    SRC_r_Cla = np.zeros((feat_tr_dic.shape[1], feat_l_te.shape[1]))
    for i in range(feat_l_te.shape[1]):
        x = cv.Variable((feat_tr_dic.shape[1],1)) #目的関数の次元
        objective = cv.Minimize(cv.norm(x,1))
        constraints = [feat_l_te[:,i,np.newaxis] == feat_tr_dic@x]
        prob = cv.Problem(objective, constraints)
        result = prob.solve()
        #classVariableから値を取るためには.valueを付ける
        SRC_Cla[:,i] = np.squeeze(x.value)
        #各クラスに関連する表現ベクトルの値を抜き出す
        #lは仕切りの値
        l = 0
        j = 0
        while j < matrix_index.shape[0]-1:
            SRC_l_Cla[l:l+matrix_index[j],i] = SRC_Cla[l:l+matrix_index[j],i]     
            l = l + matrix_index[j]
            j = j + 1
            SRC_r_Cla[l:l+matrix_index[j],i] = SRC_Cla[l:l+matrix_index[j],i]
            l = l + matrix_index[j]
            j = j + 1
    #パラメータを使って擬似的に各クラスのテスト信号の特徴ベクトルを作成
    dout_feat_l_te = np.zeros((feat_l_te.shape[0], feat_l_te.shape[1]))
    dout_feat_r_te = np.zeros((feat_l_te.shape[0], feat_l_te.shape[1])) 
    for i in range(feat_l_te.shape[1]):
        dout_feat_l_te[:,i] = feat_tr_dic@SRC_l_Cla[:,i]
    for i in range(feat_r_te.shape[1]):
        dout_feat_r_te[:,i] = feat_tr_dic@SRC_r_Cla[:,i]
    #作成した擬似特徴ベクトルとの残差を求める
    R_l =  np.zeros((feat_l_te.shape[0], feat_l_te.shape[1])) #残差を格納するための配列
    R_r =  np.zeros((feat_l_te.shape[0], feat_l_te.shape[1]))  
    for i in range(feat_l_te.shape[1]):
        R_l[:,i] = feat_l_te[:,i]-dout_feat_l_te[:,i]
        R_r[:,i] = feat_l_te[:,i]-dout_feat_r_te[:,i]
        if np.linalg.norm(R_l[:,i], ord=2) < np.linalg.norm(R_r[:,i], ord=2):
            answer[i] = 1
        elif np.linalg.norm(R_l[:,i], ord=2) > np.linalg.norm(R_r[:,i], ord=2):
            answer[i] = 2
    return answer

In [15]:
answer1 = SRC(feat_tr,feat_tr_dic_1,feat_tr_dic_2,feat_tr_dic_3,feat_tr_dic,feat_l_te,matrix_index)
answer2 = SRC(feat_tr,feat_tr_dic_1,feat_tr_dic_2,feat_tr_dic_3,feat_tr_dic,feat_r_te,matrix_index)
x1 = 0
x2 = 0
for i in range(answer1.shape[0]):
    if answer1[i] == 1:
        x1 = x1 + 1
    if answer2[i] == 2:
        x2 = x2 + 1
    temp = ((x2)/(answer2.shape[0]))*100
print(temp)

92.5
